<a href="https://colab.research.google.com/github/SepehrBazyar/QDVRP/blob/master/Quality_Driven_Vehicle_Routing_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Dependencies**

In [1]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [2]:
!pip install gurobipy pyomo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00


**Import Libraries**

In [3]:
from abc import ABC
from dataclasses import dataclass, field

import pyomo.environ as pyo

**Define Model**

In [4]:
model = pyo.ConcreteModel()

**Parameters**

In [5]:
@dataclass(unsafe_hash=True)
class Base(ABC):
  id: int

In [6]:
@dataclass(unsafe_hash=True)
class Depot(Base):
  pass

In [7]:
@dataclass(unsafe_hash=True)
class Product(Base):
  beta: float
  quality: float
  temperature: int

In [8]:
@dataclass(unsafe_hash=True)
class Vehicle(Base):
  capacity: int
  salary: float
  alpha: float
  time_window: int
  increase_rate: float = 0.005  # degrees per minute

In [9]:
@dataclass(unsafe_hash=True)
class Customer(Base):
  unload_rate: float = 0.5  # in minutes

  total_demand: int = field(
    init=False,
    hash=False,
    default=0,
  )

  @property
  def service_time(self):
    return self.unload_rate * self.total_demand

In [10]:
temperature = 25

depot = Depot(id=0)
customers = (
  c1 := Customer(id=1),
  c2 := Customer(id=2),
  c3 := Customer(id=3),
  c4 := Customer(id=4),
  c5 := Customer(id=5),
  c6 := Customer(id=6),
  c7 := Customer(id=7),
)
nodes = depot, *customers

products = (
  p1 := Product(id=1, beta=0.2, quality=0.9, temperature=5),
  p2 := Product(id=2, beta=0.5, quality=0.85, temperature=3),
)

vehicles = (
  v1 := Vehicle(id=1, capacity=30, salary=1.5, alpha=4, time_window=90),
  v2 := Vehicle(id=2, capacity=25, salary=2.5, alpha=5, time_window=120),
  v3 := Vehicle(id=3, capacity=20, salary=1.8, alpha=2, time_window=100),
)

demands = {
  (p1, c1): 3, (p1, c2): 5, (p1, c3): 2, (p1, c4): 7, (p1, c5): 4, (p1, c6): 6, (p1, c7): 8,
  (p2, c1): 2, (p2, c2): 3, (p2, c3): 1, (p2, c4): 2, (p2, c5): 4, (p2, c6): 2, (p2, c7): 3,
}
for (_, customer), demand in demands.items():
  customer.total_demand += demand

distances = {
  (depot, c1): 10,
  (depot, c2): 15,
  (depot, c3): 20,
  (depot, c4): 25,
  (depot, c5): 30,
  (depot, c6): 22,
  (depot, c7): 27,
  (c1, c2): 12, (c1, c3): 18, (c1, c4): 22, (c1, c5): 28, (c1, c6): 16, (c1, c7): 21,
  (c2, c3): 14, (c2, c4): 20, (c2, c5): 24, (c2, c6): 18, (c2, c7): 26,
  (c3, c4): 16, (c3, c5): 21, (c3, c6): 24, (c3, c7): 30,
  (c4, c5): 12, (c4, c6): 15, (c4, c7): 19,
  (c5, c6): 19, (c5, c7): 17,
  (c6, c7): 14,
}
distances.update({(j, i): d for (i, j), d in distances.items()})

times = {
  (depot, c1): 20,
  (depot, c2): 25,
  (depot, c3): 30,
  (depot, c4): 35,
  (depot, c5): 40,
  (depot, c6): 45,
  (depot, c7): 50,
  (c1, c2): 15, (c1, c3): 20, (c1, c4): 25, (c1, c5): 30, (c1, c6): 35, (c1, c7): 40,
  (c2, c3): 10, (c2, c4): 15, (c2, c5): 20, (c2, c6): 25, (c2, c7): 30,
  (c3, c4): 10, (c3, c5): 15, (c3, c6): 20, (c3, c7): 25,
  (c4, c5): 10, (c4, c6): 15, (c4, c7): 20,
  (c5, c6): 10, (c5, c7): 15,
  (c6, c7): 10,
}
times.update({(j, i): t for (i, j), t in times.items()})

**Decision Varibales**

In [11]:
model.x = pyo.Var(nodes, nodes, vehicles, domain=pyo.Binary)
model.u = pyo.Var(customers, domain=pyo.NonNegativeReals)
model.t = pyo.Var(vehicles, domain=pyo.NonNegativeReals, bounds=(0, temperature))

In [12]:
v = {(j, k): sum(model.x[i, j, k] for i in nodes if i != j) for j in customers for k in vehicles}
q = {(p, k): sum(demands.get((p, j), 0) * v[j, k] for j in customers) for p in products for k in vehicles}
t = {
  k: (
    sum(times[i, j] * model.x[i, j, k] for i in nodes for j in nodes if i != j)
    +
    sum(j.service_time * v[j, k] for j in customers)
  ) for k in vehicles
}
d = {k: sum(j.service_time * k.increase_rate * v[j, k] for j in customers) for k in vehicles}

**Objective Function**

Minimize the Total Distance Traveled & Costs

In [13]:
def objective_func(model):
  distance_cost = sum(
    distances[i, j] * model.x[i, j, k] * 1  # per unit
    for i in nodes for j in nodes for k in vehicles if i != j
  )
  salary_cost = sum(k.salary * t[k] for k in vehicles)
  cooling_cost = sum(k.alpha * (temperature - model.t[k]) for k in vehicles)
  return distance_cost + salary_cost + cooling_cost

In [14]:
model.obj = pyo.Objective(rule=objective_func, sense=pyo.minimize)

**Constraints**

Each Customer Visited Exactly Once by Exactly One Vehicle

In [15]:
def visit_once_rule(model, j: Customer):
  return sum(v[j, k] for k in vehicles) == 1

In [16]:
model.visit_once = pyo.Constraint(customers, rule=visit_once_rule)

Flow Conservation

If a Vehicle Enters a Node, It Must Leave It.

In [17]:
def flow_conservation_rule(model, j: Customer | Depot, k: Vehicle):
    come_in = sum(model.x[i, j, k] for i in nodes if i != j)
    outside = sum(model.x[j, i, k] for i in nodes if i != j)
    return come_in == outside

In [18]:
model.flow_conservation = pyo.Constraint(nodes, vehicles, rule=flow_conservation_rule)

Start & End at The Depot For Each Vehicle

In [19]:
def start_depot_rule(model, k: Vehicle):
    return sum(model.x[depot, i, k] for i in customers) <= 1


def end_depot_rule(model, k: Vehicle):
    return sum(model.x[i, depot, k] for i in customers) <= 1

In [20]:
model.start_depot = pyo.Constraint(vehicles, rule=start_depot_rule)
model.end_depot = pyo.Constraint(vehicles, rule=end_depot_rule)

Vehicle Capacity Constraints

In [21]:
def capacity_rule(model, k: Vehicle):
    return sum(q[p, k] for p in products) <= k.capacity

In [22]:
model.capacity = pyo.Constraint(vehicles, rule=capacity_rule)

Subtour Elimination

Miller-Tucker-Zemlin Formula

In [23]:
def subtour_elimination_rule(model, i: Customer, j: Customer, k: Vehicle):
    if i != j and i != depot and j != depot:
        return model.u[i] - model.u[j] + len(customers) * model.x[i, j, k] <= len(customers) - 1

    return pyo.Constraint.Skip

In [24]:
model.subtour_elimination = pyo.Constraint(customers, customers, vehicles, rule=subtour_elimination_rule)

Quality Maintaining

In [25]:
def quality_maintaining_rule(model, p: Product, k: Vehicle):
  return 1 - (p.beta * ((model.t[k] + d[k]) - p.temperature)) >= p.quality

In [26]:
model.quality_maintaining = pyo.Constraint(products, vehicles, rule=quality_maintaining_rule)

Time Window

In [27]:
def time_window_rule(model, k: Vehicle):
  return t[k] <= k.time_window

In [28]:
model.time_window = pyo.Constraint(vehicles, rule=time_window_rule)

**Solve the Model**

In [29]:
solver_name = "glpk"

In [30]:
solver = pyo.SolverFactory(solver_name)

In [31]:
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 983.1675, 'Upper bound': 983.1675, 'Number of objectives': 1, 'Number of constraints': 175, 'Number of variables': 179, 'Number of nonzeros': 1518, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '10207', 'Number of created subproblems': '10207'}}, 'Error rc': 0, 'Time': 10.285048007965088}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

**Display Results**

In [32]:
def get_path(router: dict[Customer, Customer], *, d: Depot = depot):
  order, next = [], router.get(depot, depot)
  while next != depot:
    order.append(str(next.id))
    next = router[next]

  if len(order) == 0:
    return "-"

  return " -> ".join((str(depot.id), *order, str(depot.id)))

In [33]:
for k in vehicles:
  router = {}
  for i in nodes:
    for j in nodes:
      if i != j and round(pyo.value(model.x[i, j, k])) == 1:
        router[i] = j

  t_k = round(pyo.value(model.t[k]), ndigits=4)
  print(f"Vehicle {k.id} ({t_k:.4f}C): {get_path(router, d=depot)}")

Vehicle 1 (3.2675C): 0 -> 1 -> 2 -> 0
Vehicle 2 (3.2525C): 0 -> 6 -> 7 -> 0
Vehicle 3 (3.2500C): 0 -> 5 -> 4 -> 3 -> 0


In [34]:
print(f"Total Costs: {round(model.obj(), ndigits=4)}$")

Total Costs: 983.1675$
